In [2]:
%pip install afinn

Note: you may need to restart the kernel to use updated packages.


In [6]:
from afinn import Afinn
import pandas as pd

In [5]:
#instantiate afinn
afn = Afinn()
 
#creating list sentences
news_df = ['happy','good',
         'bad','welcomed', 'umbrella']
          
# compute scores (polarity) and labels
scores = [afn.score(article) for article in news_df]
sentiment = ['positive' if score > 0
                          else 'negative' if score < 0
                              else 'neutral'
                                  for score in scores]
     
# dataframe creation
df = pd.DataFrame()
df['topic'] =  news_df
df['scores'] = scores
df['sentiments'] = sentiment
print(df)

      topic  scores sentiments
0     happy     3.0   positive
1      good     3.0   positive
2       bad    -3.0   negative
3  welcomed     2.0   positive
4  umbrella     0.0    neutral


In [10]:
afn = Afinn()

word = "bad"
print (f"Score: {afn.score(word)}")

Score: -3.0
